## Fake News Classifier Using LSTM

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
###Drop Nan Values
df=df.dropna()


In [7]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [8]:
## Get the Dependent features
y=df['label']

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.6.0'

In [52]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
messages.reset_index(inplace=True)

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\suman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [43]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[533, 1014, 4256, 4618, 4250, 2098, 1624, 4170, 1313, 3707],
 [265, 3417, 2702, 4060, 172, 4509, 418],
 [4539, 982, 1183, 1184],
 [3395, 750, 3238, 1835, 1052, 794],
 [1386, 172, 3950, 3405, 1278, 1799, 172, 1200, 1924, 4427],
 [765,
  3359,
  1712,
  3168,
  2929,
  1342,
  794,
  4952,
  3227,
  4510,
  687,
  1605,
  3665,
  4829,
  418],
 [1635, 2322, 3125, 795, 4962, 1322, 646, 449, 3059, 3059, 4160],
 [336, 846, 3903, 907, 269, 3536, 1342, 3046, 3059, 3059, 4160],
 [157, 2391, 4775, 620, 4636, 280, 682, 3131, 1342, 197],
 [3771, 1351, 135, 4351, 4428, 4514, 4825, 1222],
 [4114, 776, 3737, 3293, 1464, 1881, 2681, 4313, 4763, 267, 3852],
 [1835, 249, 4250, 280, 1342, 269],
 [4736, 4622, 1354, 3296, 3260, 3827, 1119, 4060, 2845],
 [1422, 4093, 813, 2113, 1124, 644, 4402, 3059, 3059, 4160],
 [1616, 3179, 699, 3216, 966, 3059, 3059, 4160],
 [4658, 3846, 1787, 4519, 2836, 4144, 4039, 694, 3410, 3995],
 [3482, 4037, 3417],
 [4195, 67, 3891, 3701, 1342, 4452, 1336, 418],
 [1471, 1471, 2

### Embedding Representation

In [81]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4170 1313 3707]
 [   0    0    0 ...  172 4509  418]
 [   0    0    0 ...  982 1183 1184]
 ...
 [   0    0    0 ... 3059 3059 4160]
 [   0    0    0 ... 4076 2723  164]
 [   0    0    0 ... 3937 1837 2236]]


In [105]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  533,
       1014, 4256, 4618, 4250, 2098, 1624, 4170, 1313, 3707])

In [53]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [83]:
len(embedded_docs),y.shape

(18285, (18285,))

In [84]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [86]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [106]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 4s 347us/sample - loss: 0.0041 - accuracy: 0.9991 - val_loss: 0.6781 - val_accuracy: 0.9130
Epoch 2/10
12250/12250 [==============================] - 3s 261us/sample - loss: 0.0030 - accuracy: 0.9989 - val_loss: 0.5203 - val_accuracy: 0.9102
Epoch 3/10
12250/12250 [==============================] - 4s 293us/sample - loss: 0.0038 - accuracy: 0.9990 - val_loss: 0.6349 - val_accuracy: 0.9062
Epoch 4/10
12250/12250 [==============================] - 3s 276us/sample - loss: 0.0037 - accuracy: 0.9989 - val_loss: 0.7011 - val_accuracy: 0.9052
Epoch 5/10
12250/12250 [==============================] - 3s 258us/sample - loss: 0.0016 - accuracy: 0.9998 - val_loss: 0.7310 - val_accuracy: 0.9089
Epoch 6/10
12250/12250 [==============================] - 4s 307us/sample - loss: 0.0013 - accuracy: 0.9996 - val_loss: 0.7530 - val_accuracy: 0.9092
Epoch 7/10
12250/12250 [===========================

### Adding Dropout 

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [107]:
y_pred=model.predict_classes(X_test)

In [108]:
from sklearn.metrics import confusion_matrix

In [109]:
confusion_matrix(y_test,y_pred)

array([[3140,  279],
       [ 278, 2338]], dtype=int64)

In [110]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.907705053852527